In [ ]:
pip install imutils

In [ ]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import display
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imutils import paths
import argparse
import imutils
from sklearn.model_selection import StratifiedKFold
from statistics import mean
from sklearn.metrics import *
from sklearn import metrics

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The following part of processing the images where the images are flattened and converted to color histograms are referred from the document "https://www.pyimagesearch.com/2016/08/08/k-nn-classifier-for-image-classification/". This link has been pu in the references of our paper

In [ ]:
def image_to_feature_vector(image, size=(32, 32)):
    # resize the image to a fixed size, then flatten the image into
    # a list of raw pixel intensities
    return cv2.resize(image, size).flatten()

In [ ]:
def extract_color_histogram(image, bins=(8, 8, 8)):
    # extract a 3D color histogram from the HSV color space using
    # the supplied number of `bins` per channel
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,[0, 180, 0, 256, 0, 256])
    
    if imutils.is_cv2():
        hist = cv2.normalize(hist)
    
    else:
        cv2.normalize(hist, hist)
    # return the flattened histogram as the feature vector
    return hist.flatten()

In [ ]:
imagePaths = []

for dirname, _, filenames in os.walk('/kaggle/input/covid19-radiography-database/COVID-19 Radiography Database/'):
    for filename in filenames:
        if (filename[-3:] == 'png'):
            imagePaths.append(os.path.join(dirname, filename))

In [ ]:
image = cv2.imread(imagePaths[0])
print(image.shape)
plt.imshow(image)
plt.show()

In [ ]:
rawImages = []
features = []
labels = []
img=[]

for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath)
    # extract raw pixel intensity "features", followed by a color
    # histogram to characterize the color distribution of the pixels
    # in the image
    pixels = image_to_feature_vector(image)
    hist = extract_color_histogram(image)
    rawImages.append(pixels)
    features.append(hist)
    labels.append(label)
    img.append(image)
   
print(labels[0])

In [ ]:
rawImages = np.array(rawImages)
features = np.array(features)
labels = np.array(labels)
print("[INFO] pixels matrix: {:.2f}MB".format(rawImages.nbytes / (1024 * 1000.0)))
print("[INFO] features matrix: {:.2f}MB".format(features.nbytes / (1024 * 1000.0)))

In [ ]:
from sklearn.naive_bayes import GaussianNB
nbacc=[]
nbrecall=[]
stratifiedcv= StratifiedKFold()
stratifiedcv.get_n_splits(rawImages, labels)
for train_ix, test_ix in stratifiedcv.split(rawImages, labels):
    trainRI2, testRI2 = rawImages[train_ix], rawImages[test_ix]
    trainRL2, testRL2 = labels[train_ix], labels[test_ix]
    model=GaussianNB()
    model.fit(trainRI2, trainRL2)
    acc = model.score(testRI2, testRL2)
    yhat=model.predict(testRI2)
    accround=round(acc,2)
    accuracy=accuracy_score(testRL2, yhat)
    recall= recall_score(testRL2, yhat, average='macro')
    nbacc.append(accuracy)
    nbrecall.append(recall)
    print("Raw pixel accuracy for Naive Bayes is: {:.2f}%".format(accuracy * 100))
    print("Raw pixel recall for Naive Bayes is: {:.2f}%".format(recall * 100))
print("Average Raw pixel accuracy for Naive Bayes is: {:.2f}%".format(mean(nbacc) * 100))
print("Average Raw pixel recall for Naive Bayes is: {:.2f}%".format(mean(nbrecall) * 100))

In [ ]:
fig2 = plt.figure(figsize =(9, 6))
folds=[1,2,3,4,5]
plt.plot(folds,nbacc, label='accuracy')
plt.plot(folds,nbrecall, label='recall')
plt.xlabel('folds')
plt.ylabel('accuracies and recall')
plt.title('Naive Bayes Raw pixel accuracy and recall obtained at different folds ')
plt.legend(loc='upper left')
plt.show()

In [ ]:
nbacc2=[]
nbrecall2=[]
stratifiedcv= StratifiedKFold()
stratifiedcv.get_n_splits(features, labels)
for train_ix, test_ix in stratifiedcv.split(features, labels):
    trainFeat2, testFeat2 = features[train_ix], features[test_ix]
    trainLabels2, testLabels2 = labels[train_ix], labels[test_ix]
    modelNB=GaussianNB()
    modelNB.fit(trainFeat2, trainLabels2)
    acc = modelNB.score(testFeat2, testLabels2)
    yhat=modelNB.predict(testFeat2)
    accround=round(acc,2)
    accuracy=accuracy_score(testLabels2, yhat)
    recall= recall_score(testLabels2, yhat, average='macro')
    nbacc2.append(accuracy)
    nbrecall2.append(recall)
    print("Histogram accuracy for Naive Bayes is: {:.2f}%".format(accuracy * 100))
    print("Histogram recall for Naive Bayes is: {:.2f}%".format(recall * 100))
print("Histogram pixel accuracy for Naive Bayes is: {:.2f}%".format(mean(nbacc2) * 100))
print("Histogram pixel recall for Naive Bayes is: {:.2f}%".format(mean(nbrecall2) * 100))


In [ ]:
fig2 = plt.figure(figsize =(9, 6))
folds=[1,2,3,4,5]
plt.plot(folds,nbacc2, label='accuracy')
plt.plot(folds,nbrecall2, label='recall')
plt.xlabel('folds')
plt.ylabel('accuracies and recall')
plt.title('Histogram accuracy and recall obtained at different folds ')
plt.legend(loc='upper left')
plt.show()